### Mie Theory

### Mie theory coefficients and Bessell functions (JJF)

There are 4 types of coefficients for the Mie theory expansion for the electric field in the vicinity of a spherical particle: $a_n$ and $b_n$ for the scattered fields (external to the particle), and $c_n$ and $d_n$ for the fields inside the particle (internal fields).  These coefficients can be expressed as follows:
\begin{equation}
a_n = \frac{m^2 j_n(mx)[x j_n(x)]' - \mu_1 j_n(x) [mxj_n (mx)]'}{m^2 j_n(mx)[x h_n^{(1)}(x)]' - \mu_1 h_n^{(1)}(x) [mxj_n (mx)]'} \\
\end{equation}

\begin{equation}
b_n = \frac{\mu_1 j_n(mx)[x j_n(x)]' - j_n(x) [mxj_n (mx)]'}
{\mu_1 j_n(mx)[x h_n^{(1)}(x)]' -  h_n^{(1)}(x) [mxj_n (mx)]'}  \\
\end{equation}

\begin{equation}
c_n = \frac{\mu_1 j_n(mx)[x h_n^{(1)}(x)]' - \mu_1 h_n^{(1)}(x) [xj_n (mx)]'}{\mu_1 j_n(mx)[x h_n^{(1)}(x)]' - h_n^{(1)}(x) [mxj_n (mx)]'} \\
\end{equation}

\begin{equation}
d_n = \frac{\mu_1 m j_n(mx)[x h_n^{(1)}(x)]' - \mu_1 m h_n^{(1)}(x) [mxj_n (mx)]'}
{m^2 j_n(mx)[x h_n^{(1)}(x)]' - \mu_1 h_n^{(1)}(x) [mxj_n (mx)]'} \\
\end{equation}.

In the above, $m = n_{sphere}/n_{medium}$ represents the refractive index of the spherical particle divided by the refractive index of the surrounding, $x = 2\pi r/\lambda$ is the size parameter of the sphere, where $r$ is the radius of the sphere and $\lambda$ is the wavelength of light, and $\mu_1 = \mu_{sphere}/\mu_{medium}$ represents the magnetic permiability of the spherical particle divided by the permeability of the medium.  The functions $j_n(z)$ and $h_n^{(1)}(z) = j_n(z) + iy_n(z)$ are the spherical Bessel functions.  The spherical Bessel functions are defined in terms of the Bessel functions of the first kind
\begin{equation}
j_n(z) = \sqrt{\frac{\pi}{2z}} J_{n+0.5}(z)
\end{equation}
and second kind
\begin{equation}
y_n(z) = \sqrt{\frac{\pi}{2z}} Y_{n+0.5}(z).
\end{equation}
The derivative terms are defined as
\begin{equation}
[z j_n(z)]' = zj_{n-1}(z) - nj_n(z)
\end{equation}
and
\begin{equation}
[zh_n^{(1)}(z)]' = zh_{n-1}^{(1)}(z) - nh_{n}^{(1)}(z).
\end{equation}


Let's test the Bessel and Spherical Bessel functions using the relations here and the built-in functions in scipy.



In [1]:
### Start running here!
from scipy.special import spherical_jn
from scipy.special import spherical_yn
from scipy.special import jv
from scipy.special import yv
import numpy as np
#import miepython
from matplotlib import pyplot as plt

### define size parameter
# radius = 10 nm
r = 100e-9
# wavelength = 500 nm
lam = 500e-9

# ri of sphere
ns = 1.5
# ri of medium
nm = 1.0
# mu of sphere
mus = 1.0
# mu of surrounding
mum = 1.0

### parameters
m = ns/nm 
mu = mus/mum
x = 2*np.pi*r/lam

Let's define the Spherical Bessel functions and also the derivative terms here!

In [2]:

def s_jn(n,z):
  ''' Compute the spherical bessel function from the Bessel function
      of the first kind
  '''
  ns = n+0.5
  return np.sqrt(np.pi/(2*z)) * jv(ns, z)

def s_yn(n,z):
  ''' Compute the spherical bessel function from the Bessel function
      of the second kind
  '''
  ns = n+0.5
  return np.sqrt(np.pi/(2*z)) * yv(ns, z)

def s_hn(n, z):
  ''' Compute the spherical bessel function h_n^{(1)} '''
  ci = 0+1j
  return spherical_jn(n, z) + ci*spherical_yn(n, z)

def z_jn_prime(n,z):
  ''' Compute derivative of z*j_n(z) using recurrence relations '''
  return z*spherical_jn(n-1, z)-n*spherical_jn(n,z)

def z_hn_prime(n,z):
  ''' Compute derivative of z*h_n^{(1)}(z) using recurrence relations '''
  return z*s_hn(n-1,z) -n*s_hn(n,z)

def mie_coeff(n, m, mu, x):
  ''' computes the Mie theory a, b, c, and d coefficients for a
      given order n, relative refractive index m, relative permeability mu, and
      size parameter x.  Returns coefficients as an array. '''
  
  ### pre-compute the different Bessel function terms that will be used over and over again
  ## jnx <==> j_n(x)
  jnx = spherical_jn(n,x)
  ## jnmx <==> j_n(mx)
  jnmx = spherical_jn(n,m*x)
  ## hnx <==> h_n^{(1)}(x)
  hnx = s_hn(n, x)
  ## xjnxp <==> [x*j_n(x)]'
  xjnxp = z_jn_prime(n, x)
  ## mxjnmxp <==> [mx*j_n(mx)]'
  mxjnmxp = z_jn_prime(n, m*x)
  ## xhnxp <==> [x*h_n^{(1)}(x)]'
  xhnxp = z_hn_prime(n, x)

  ### now use pre-computed values to compute coefficients!
  ## a_n first
  a_num = m**2 * jnmx * xjnxp - mu * jnx * mxjnmxp 
  a_den = m**2 * jnmx * xhnxp - mu * hnx * mxjnmxp
  a_n = a_num / a_den

  ## b_n second
  b_num = mu * jnmx * xjnxp - jnx * mxjnmxp 
  b_den = mu * jnmx * xhnxp - hnx * mxjnmxp
  b_n = b_num / b_den

  ## c_n third
  c_num = mu * jnx * xhnxp - mu * hnx * xjnxp
  c_den = mu * jnmx * xhnxp - hnx * mxjnmxp
  c_n = c_num / c_den

  ## d_n fourth
  d_num = mu * m * jnx * xhnxp - mu * m * hnx * xjnxp 
  d_den = m**2 * jnmx * xhnxp - mu * hnx * mxjnmxp
  d_n = d_num / d_den
  return [a_n, b_n, c_n, d_n]

In [3]:
# make an array of coefficients consistent 
# with the glass sphere of radius 100e-9 m in air 
# illuminated with lambda = 500e-9 m light
_n_max = int(x + 4*x**(1/3.)+2)
_n_array = np.copy( np.linspace(1, _n_max, _n_max, dtype=int) )


print(_n_array)

#print(s_jn(_n_array, x))

#print(s_yn(_n_array, x))

#print(s_hn(n, x))


#print(z_jn_prime(n, x))

#print(z_hn_prime(n, x))

#print(mie_coeff(n, m, mu, x))


[1 2 3 4 5 6 7]
